### Determining the optimal number of hidden layers and neurons for an Artificial Neural Network (ANN) 
This can be challenging and often requires experimentation. However, there are some guidelines and methods that can help you in making an informed decision:

- Start Simple: Begin with a simple architecture and gradually increase complexity if needed.
- Grid Search/Random Search: Use grid search or random search to try different architectures.
- Cross-Validation: Use cross-validation to evaluate the performance of different architectures.
- Heuristics and Rules of Thumb: Some heuristics and empirical rules can provide starting points, such as:
  -    The number of neurons in the hidden layer should be between the size of the input layer and the size of the output layer.
  -  A common practice is to start with 1-2 hidden layers.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

ModuleNotFoundError: No module named 'tensorflow.keras.wrappers.scikit_learn'

In [2]:
data=pd.read_csv('Churn_Modelling.csv')

data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data.head()


label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])


ohe_geo=OneHotEncoder(sparse_output=False)
geo_encoded=ohe_geo.fit_transform(data[['Geography']])
geo_encoded_df=pd.DataFrame(geo_encoded,columns=ohe_geo.get_feature_names_out(['Geography']))
geo_encoded_df


## combine the encoded geography columns with the original data
data = pd.concat([data.drop('Geography',axis=1), geo_encoded_df], axis=1)


## now split the data into features and targets
X = data.drop('Exited',axis=1)
Y = data['Exited']

## split the data in training and testing sets
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

## scale these features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

# dump the encoders and scaler
with open('label_encoder_gender.pkl', 'wb') as file:
     pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(ohe_geo,file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler,file)



In [ ]:
## define a function to create a model and try diffrent parameters(KerasClassifier)

def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1])))
    
    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))

    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

    return model

## create  a Keras classifier
model=KerasClassifier(layers=1,neurons=32,build_fn=create_model,epochs=50,batch_size=10,verbose=0)


# Define the grid search parameters
param_grid = {
    'neurons': [16, 32, 64, 128],
    'layers': [1, 2],
    'epochs': [50, 100]
}

# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3,verbose=1)
grid_result = grid.fit_transform(X_train, Y_train)

# Print the best parameters
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))




Fitting 3 folds for each of 16 candidates, totalling 48 fits


ValueError: 
All the 48 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
48 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Silajit Chakraborty\AppData\Roaming\Python\Python312\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Silajit Chakraborty\AppData\Roaming\Python\Python312\site-packages\scikeras\wrappers.py", line 1501, in fit
    super().fit(X=X, y=y, sample_weight=sample_weight, **kwargs)
  File "C:\Users\Silajit Chakraborty\AppData\Roaming\Python\Python312\site-packages\scikeras\wrappers.py", line 770, in fit
    self._fit(
  File "C:\Users\Silajit Chakraborty\AppData\Roaming\Python\Python312\site-packages\scikeras\wrappers.py", line 925